# Getting Started: Building Agents over LlamaCloud RAG Pipelines

<a href="https://colab.research.google.com/github/run-llama/llamacloud-demo/blob/main/examples/getting_started_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this notebook we show you how to build a function calling agent (powered by OpenAI) over RAG pipelines built with LlamaCloud.

Adding an agentic layer to RAG allows you to build in a layer of query planning and state management that allows you to ask multi-part complex questions over existing RAG pipelines and get back answers in a conversational manner.


## Initial Setup 

Let's start by importing some simple building blocks.  

The main thing we need is:
1. the OpenAI API (using our own `llama_index` LLM class)
2. a place to keep conversation history 
3. a definition for tools that our agent can use.

If you're opening this Notebook on colab, you will probably need to install LlamaIndex 🦙.


In [3]:
%pip install -U llama-index-indices-managed-llama-cloud
%pip install -U llama-index
%pip install -U llama-index-core


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
import nest_asyncio
nest_asyncio.apply()

from IPython.display import Markdown, display

Make sure your OPENAI_API_KEY is set. Otherwise explicitly specify the `api_key` parameter.

In [5]:
from llama_index.llms.openai import OpenAI
import os

os.environ["OPENAI_API_KEY"] = "<openai_api_key>"

llm = OpenAI(model="gpt-4.1")

## Build Two LlamaCloud Indexes

Our data sources are the 2021 Lyft and Uber 10K documents.

In contrast to the other getting started examples, in this notebook we will build **two** RAG pipelines: one for Uber and one for Lyft. This is for the sake of example; we can plug in both RAG pipelines as tools for the agent to reason over.

To create each index, follow the instructions:

In [24]:
from llama_cloud import (
    LlamaParseParameters,
)

# Transformation auto config
transform_config = {
    'mode': 'auto',
    'config': {
        'chunk_size': 1024, # editable
        'chunk_overlap': 20 # editable
    }
}

## LLamaParse Parameters
llama_parse_parameters = LlamaParseParameters(
    premium_mode=True
)

In [25]:
from llama_index.indices.managed.llama_cloud import LlamaCloudIndex

# Uber Index
index_uber = LlamaCloudIndex.create_index(
  name="<index_uber>", 
  project_name="<project_name>",
  api_key="<api_key>",
  organization_id="<organization_id>",
  transform_config=transform_config,
  llama_parse_parameters=llama_parse_parameters
)

# Lyft index
index_lyft = LlamaCloudIndex.create_index(
  name="<index_lyft>", 
  project_name="<project_name>",
  api_key="<api_key>",
  organization_id="<organization_id>",
  transform_config=transform_config,
  llama_parse_parameters=llama_parse_parameters
)

In [26]:
index_uber.upload_file_from_url(
    file_name="uber_2021.pdf",
    url="https://www.dropbox.com/s/te0a2w227v27iag/uber_2021.pdf?dl=1",
    verbose=True
)

index_lyft.upload_file_from_url(
    file_name="lyft_2021.pdf",
    url="https://www.dropbox.com/s/qctkz6nxhm0y5qe/lyft_2021.pdf?dl=1",
    verbose=True
)

# You can also use
# file_id = index.upload_file(temp_file_path, verbose=True)
# assert file_id is not None

Uploaded file 0db37c97-84c7-444e-a7a4-6e682f557be3 with ID 0db37c97-84c7-444e-a7a4-6e682f557be3
Loading file
File ingestion finished for 0db37c97-84c7-444e-a7a4-6e682f557be3
Done!
Syncing pipeline ed985a02-9897-4b4b-a8aa-2516ec959dc2
Done!
Uploaded file e77ff085-0300-459f-87d5-595b5fb03994 with ID e77ff085-0300-459f-87d5-595b5fb03994
Loading file
File ingestion finished for e77ff085-0300-459f-87d5-595b5fb03994
Done!
Syncing pipeline efbb70c5-2d18-4afb-92e1-5ec7edd885c9
Done!


'e77ff085-0300-459f-87d5-595b5fb03994'

For each index, get a query engine from the index, which gives us an out-of-the-box RAG pipeline.

In [27]:
query_engine_uber = index_uber.as_query_engine(llm=llm)
query_engine_lyft = index_lyft.as_query_engine(llm=llm)

## OpenAI Agent over LlamaCloud RAG Pipelines

We convert both query engines to tools and pass it to a function calling OpenAI agent.

In [28]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata

tool_uber = QueryEngineTool(
    query_engine=query_engine_uber,
    metadata=ToolMetadata(
        name="uber_10k",
        description=(
            "Provides information about Uber financials for year 2021. "
            "Use a detailed plain text question as input to the tool."
        ),
    ),
)
tool_lyft = QueryEngineTool(
    query_engine=query_engine_lyft,
    metadata=ToolMetadata(
        name="lyft_10k",
        description=(
            "Provides information about Lyft financials for year 2021. "
            "Use a detailed plain text question as input to the tool."
        ),
    ),
)

In [29]:
from llama_index.core.agent.workflow import FunctionAgent

agent = FunctionAgent(
    tools=[tool_uber, tool_lyft],
    llm=llm,
    system_prompt="You are a helpful assistant that can answer questions about Uber and Lyft.",
    verbose=True,
)

In [30]:
response = await agent.run(user_msg="Tell me both the tailwinds for Uber and Lyft?")

Running step init_run
Step init_run produced event AgentInput
Running step setup_agent
Step setup_agent produced event AgentSetup
Running step run_agent_step
Step run_agent_step produced event AgentOutput
Running step parse_agent_output
Step parse_agent_output produced no event
Running step call_tool
Running step call_tool
Step call_tool produced event ToolCallResult
Running step aggregate_tool_results
Step aggregate_tool_results produced no event
Step call_tool produced event ToolCallResult
Running step aggregate_tool_results
Step aggregate_tool_results produced event AgentInput
Running step setup_agent
Step setup_agent produced event AgentSetup
Running step run_agent_step
Step run_agent_step produced event AgentOutput
Running step parse_agent_output
Step parse_agent_output produced event StopEvent


In [31]:
display(Markdown(f"{response}"))

Here are the main tailwinds for both Uber and Lyft in 2021:

Uber Tailwinds:
1. Significant Growth in Gross Bookings and Revenue: Gross Bookings rose by 56% and revenue by 57%, driven by both Delivery and Mobility segments.
2. Expansion of Delivery Business: Delivery Gross Bookings grew by 66%, fueled by increased demand for food delivery and expansion into new markets.
3. Recovery in Mobility Segment: Mobility Gross Bookings increased by 36% as pandemic restrictions eased and travel resumed.
4. Strategic Acquisitions: Key acquisitions (Transplace, Drizly, Cornershop) expanded Uber’s offerings in freight, alcohol, and grocery delivery.
5. Platform Synergies and Membership Programs: Launch of Uber One and growth in membership programs led to higher user engagement.
6. Operational and Technological Strengths: Proprietary technology and a large network enabled efficiency and rapid scaling.
7. Improved Financial Performance: Net loss decreased by 93% and Adjusted EBITDA improved, reflecting better cost management.

Lyft Tailwinds:
1. Recovery from the COVID-19 Pandemic: Demand rebounded as vaccines rolled out and communities reopened, with a 36% revenue increase and 49.2% more Active Riders in Q4 2021.
2. Increased Revenue per Active Rider: All-time high revenue per rider, driven by more frequent rides and a shift to higher-value rides like airport trips.
3. Strategic Investments and Innovation: Continued investment in R&D, acquisitions, and expansion of multimodal offerings (bikes, scooters, rentals).
4. Brand Strength and Social Responsibility: Focus on values, environmental goals (100% EVs by 2030), and social responsibility fostered loyalty.
5. Financial Strength: Ended 2021 with $2.3 billion in cash and investments, providing strong liquidity.
6. Profitability Milestone: Achieved first annual Adjusted EBITDA profitability.

Both companies benefited from the recovery in travel demand, strategic investments, and operational improvements, positioning them for continued growth.

In [32]:
response = await agent.run(user_msg="What are the investments made by Uber?")

Running step init_run
Step init_run produced event AgentInput
Running step setup_agent
Step setup_agent produced event AgentSetup
Running step run_agent_step
Step run_agent_step produced event AgentOutput
Running step parse_agent_output
Step parse_agent_output produced no event
Running step call_tool
Step call_tool produced event ToolCallResult
Running step aggregate_tool_results
Step aggregate_tool_results produced event AgentInput
Running step setup_agent
Step setup_agent produced event AgentSetup
Running step run_agent_step
Step run_agent_step produced event AgentOutput
Running step parse_agent_output
Step parse_agent_output produced event StopEvent


In [33]:
display(Markdown(f"{response}"))

In 2021, Uber made several significant investments across different categories:

1. Acquisitions:
   - Transplace: Uber acquired 100% of Transplace, a major logistics and transportation management provider, to expand its Uber Freight business.
   - Cornershop: Uber completed the acquisition of the remaining ownership interest in Cornershop, a grocery delivery platform.
   - Drizly: Uber acquired Drizly, an on-demand alcohol marketplace, to enhance its Delivery business.

2. Equity Investments and Related Transactions:
   - Moove: Uber invested in Moove, a vehicle fleet operator in Spain, acquiring a 30% minority interest and providing a term loan, with potential for further investment.
   - Aurora: Uber received equity in Aurora Innovation, Inc. after selling its ATG (autonomous driving) business to Aurora. Uber’s holdings became publicly traded after Aurora’s SPAC merger.
   - Didi: Uber’s investment in Didi became a marketable equity security after Didi’s IPO.
   - Zomato: Uber’s investment in Zomato became a marketable equity security after Zomato’s IPO in India.

3. Other Financial Investments:
   - Uber invested in marketable securities ($1.1 billion), non-marketable equity securities ($982 million), and notes receivable ($297 million).

4. Divestitures and Related Investments:
   - Aurora: Uber sold its ATG business to Aurora and received equity as part of the deal.
   - Yandex: Uber sold portions of its interests in MLU B.V. and Yandex Self Driving Group B.V., as well as certain delivery businesses, to Yandex.

These investments reflect Uber’s strategy to expand its core businesses, invest in technology and mobility companies, and manage a portfolio of financial assets.

In [34]:
response = await agent.run(user_msg="Compare the investments made by Uber and lyft?")

Running step init_run
Step init_run produced event AgentInput
Running step setup_agent
Step setup_agent produced event AgentSetup
Running step run_agent_step
Step run_agent_step produced event AgentOutput
Running step parse_agent_output
Step parse_agent_output produced no event
Running step call_tool
Running step call_tool
Step call_tool produced event ToolCallResult
Running step aggregate_tool_results
Step aggregate_tool_results produced no event
Step call_tool produced event ToolCallResult
Running step aggregate_tool_results
Step aggregate_tool_results produced event AgentInput
Running step setup_agent
Step setup_agent produced event AgentSetup
Running step run_agent_step
Step run_agent_step produced event AgentOutput
Running step parse_agent_output
Step parse_agent_output produced event StopEvent


In [35]:
display(Markdown(f"{response}"))

Here’s a comparison of the investments made by Uber and Lyft in 2021:

Uber

- Acquisitions:
  - Acquired the remaining 45% of Cornershop Cayman (grocery delivery).
  - Acquired Drizly (on-demand alcohol marketplace) to expand its Delivery business.
  - Acquired Transplace (logistics and freight management) to grow Uber Freight.

- Capital Expenditures:
  - Spent $298 million on property and equipment.

- Strategic Investments:
  - Invested $1.1 billion in marketable securities and $982 million in non-marketable equity securities.
  - Made $297 million in purchases of notes receivable.
  - Sold equity interests in several companies (e.g., MLU B.V., Yandex Self Driving Group, Yandex.Eats, Yandex.Lavka, Yandex.Delivery).
  - Recognized gains from investments in companies like Grab, Aurora, and Zomato, but also had losses (notably on Didi).

- Focus: Uber’s investments were aimed at expanding its core businesses (mobility, delivery, and freight), enhancing logistics, and making substantial financial investments in both public and private companies.

Lyft

- Acquisitions and Strategic Initiatives:
  - Invested heavily in research and development and completed multiple strategic acquisitions to enhance its platform.
  - Focused on expanding its bikeshare programs, committing $100 million in New York (with $87.1 million invested by end of 2021) and significant investments in Chicago’s Divvy program ($23.1 million in payments and $23.5 million in investments by end of 2021).
  - Expanded its network of shared bikes and scooters (“Light Vehicles”).
  - Invested in third-party self-driving technology through Lyft Autonomous.

- Capital Expenditures:
  - Made focused capital expenditures to support bikeshare, mobility, and technology initiatives.
  - Maintained future purchase commitments totaling $120.3 million as of December 31, 2021.

- Environmental and Regulatory Investments:
  - Invested in compliance with environmental goals (e.g., 100% electric vehicles by 2030).
  - Planned further investments to meet regulatory requirements like the California Clean Miles Standard.

- Focus: Lyft’s investments centered on expanding its mobility network (especially bikeshare and scooters), advancing autonomous vehicle technology, and meeting environmental and regulatory commitments.

Summary

- Uber’s investments were broader, including major acquisitions in delivery and logistics, significant financial investments, and capital expenditures.
- Lyft’s investments were more focused on mobility innovation (bikeshare, scooters, autonomous vehicles), environmental goals, and regulatory compliance, with substantial capital directed toward bikeshare infrastructure.

If you need more details on any specific investment or area, let me know!